In [156]:
from utils.utils import get_google_play_reviews
import numpy as np
import pandas as pd


from dotenv import load_dotenv
import os

import google.generativeai as genai


import yaml
from utils import utils

import time

In [157]:
load_dotenv()

True

**IMPORTANT**: you need to create a Google Gemini API key and save it in ".env" file located in the root directory.

In [158]:
GEMINI_API_KEY=os.getenv("GEMINI_NEW")

## Ingest reviews from Google Playstore

In [159]:
apps = ['com.binance.dev', 'co.mona.android', 'com.bybit.app']
crypto_com = apps[1]
bybit = apps[2]

In [160]:
n_samples=60
res, token = get_google_play_reviews(crypto_com, count=n_samples)
re2, token = get_google_play_reviews(bybit, count=n_samples)

In [161]:
df = pd.DataFrame(res)

## Generate List of Topics

In [162]:
with open("prompts/prompts.yaml") as file:
    try:
        prompts = yaml.safe_load(file)
    except yaml.YAMLError as exc:
        print(exc)

In [163]:
prompt = prompts["prompt_v9a"].format(industry="Crypto")

In [164]:
print(prompt)

You are an expert Customer Success Manager working int the Crypto industry. Can you list the most common categories that could exist for a mobile app within this industry?Group similar categories into one, i.e: Customer Support and Customer Services should be grouped under Customer Support.Provide your answer as a list, eg: [category1, category2,..]


In [165]:
res = utils.gemini_query(prompt, gemini_key = GEMINI_API_KEY)

In [166]:
category_list = res.strip("[|]").split(",")

In [167]:
category_list

['Wallet Management',
 ' Trading & Investing',
 '  DeFi & Yield Farming',
 ' NFT Management & Marketplace',
 '  Staking & Governance',
 '  News & Market Data',
 ' Security & Compliance Tools',
 '  Education & Learning',
 '  Social & Community',
 '  Customer Support',
 ' Regulatory Compliance Reporting',
 ' Analytics & Portfolio Tracking']

In [168]:
with open("data/1.crypto_category.txt", "r") as f:
    category_list = f.readlines()

In [169]:
category_list = [cat.strip() for cat in category_list[0].split(",")]

In [170]:
category_list

['Security',
 'Usability/UI/UX',
 'Transaction Fees/Speed',
 'Customer Support',
 'Features/Functionality',
 'Account Management',
 'Educational Resources/Onboarding',
 'Wallet Security/Integration',
 'Privacy',
 'Reliability/Stability',
 'Customer Service',
 'Verification/KYC/AML Processes']

## Tag reviews

**Note**: there is a **debug** variable that is currently set up to **True**. This will make the code tag a sample of 20 reviews. If you want to run this against the total reviews, change the vairable to **False**.

In [171]:
test_category = category_list
perc = df.shape[0]//10
topics_gemini = []

In [172]:
debug=True
start_time = time.time()
for i, review in enumerate(df.content):
    prompt2 = prompts["prompt_v9b"].format(industry="Crypto", 
                                                 categories=category_list,
                                                input_text = review )
    topic = utils.gemini_query(prompt2, gemini_key = GEMINI_API_KEY, debug=debug)
    topics_gemini.append(topic)
    if i%perc == 0:
        print(f"{i} out of {len(reviews)} done.")
    

end_time = time.time()
print(f"{i} reviews were processed in {(end_time-start_time)/60} minutes")

0 out of 30 done.
6 out of 30 done.
12 out of 30 done.
18 out of 30 done.
Gemini Failed to respond. Sleeping...
Entering recursive step. 1
Gemini Failed to respond. Sleeping...
Entering recursive step. 1
24 out of 30 done.
30 out of 30 done.
36 out of 30 done.
Gemini Failed to respond. Sleeping...
Entering recursive step. 1
Gemini Failed to respond. Sleeping...
Entering recursive step. 1
42 out of 30 done.
48 out of 30 done.
54 out of 30 done.
Gemini Failed to respond. Sleeping...
Entering recursive step. 1
Gemini Failed to respond. Sleeping...
Entering recursive step. 1
59 reviews were processed in 3.002107620239258 minutes


In [173]:
df["gemini_llm_topic"] = topics_gemini

In [174]:
df_o = df.tail(30)
df_n = df.head(30)

Lets look at a few reviews.

In [175]:
for row in range(df.head(10).shape[0]):
        print("App: ", crypto_com, " Tagged Topic: ", df.iloc[row]["gemini_llm_topic"])
        print("Review Text:")
        print()
        print(df_c.iloc[row]["content"])
        print("***************************")
        print()

App:  co.mona.android  Tagged Topic:  Generic feedback
Review Text:

the bugs seemed to have been fixed but pls make it possible to access customer service within the app
***************************

App:  co.mona.android  Tagged Topic:  Usability/UI/UX
Review Text:

can't unlock app
***************************

App:  co.mona.android  Tagged Topic:  Usability/UI/UX
Review Text:

selling people Info
***************************

App:  co.mona.android  Tagged Topic:  Generic feedback
Review Text:

Wild&crasy
***************************

App:  co.mona.android  Tagged Topic:  Features/Functionality
Review Text:

great app. easy to use for a beginner.
***************************

App:  co.mona.android  Tagged Topic:  Account Management
Review Text:

I purchased a brand new crypto coin today, Walrus (WAL) but I'm not able to find a lot of crypto coins here for some reason unfortunately even though they are popular or brand new, newly released crypto coins.
***************************

App:  c

## Train a neural net

In [176]:
from utils.utils import TopicNeuralNet

In [177]:
testmodel = TopicNeuralNet(df_o.content, df_o.gemini_llm_topic)

### With no preprocessing and no bootstraping

In [179]:
start = time.time()
testmodel.fit(bootstrapping=False, preprocess=False, epochs=20, factor=100)
end = time.time()
print("Elapsed time: ", np.round((end-start)/60,2),"minutes.")

Bootstrapping by a factor of 1
Normalizing text
Fitting text tokenizer
Fitting label tokenizer
Tokenizing text
Tokenizing labels
Passing text sequences
Setting model architecture
Compiled model
Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.2500 - loss: 2.1898 - val_accuracy: 0.1154 - val_loss: 2.1960
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.2500 - loss: 2.1423 - val_accuracy: 0.1154 - val_loss: 2.1939
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.5000 - loss: 2.1537 - val_accuracy: 0.1154 - val_loss: 2.1959
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.5000 - loss: 2.0485 - val_accuracy: 0.1154 - val_loss: 2.2002
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.5000 - loss: 1.9483 - val_accuracy: 0.1154 - val_loss: 2.2071
Epoch 5: early stopping
Elapsed time:  0.07 minutes.


In [180]:
y_hat_old = testmodel.predict(df_o.content)
y_hat_new = testmodel.predict(df_o.content)

Normalizing text
Tokenizing text
Passing text sequences
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
Normalizing text
Tokenizing text
Passing text sequences
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


### With preprocessing and no bootstraping

In [181]:
start = time.time()
testmodel.fit(bootstrapping=False, preprocess=True, epochs=20, factor=100)
end = time.time()
print("Elapsed time: ", np.round((end-start)/60,2),"minutes.")

Bootstrapping by a factor of 1
Normalizing text
Fitting text tokenizer
Fitting label tokenizer
Tokenizing text
Tokenizing labels
Passing text sequences
Setting model architecture
Compiled model
Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 2.1929 - val_accuracy: 0.1154 - val_loss: 2.1877
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.2500 - loss: 2.1533 - val_accuracy: 0.1154 - val_loss: 2.1773
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.5000 - loss: 2.0681 - val_accuracy: 0.1154 - val_loss: 2.1705
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.5000 - loss: 2.0466 - val_accuracy: 0.1154 - val_loss: 2.1703
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.5000 - loss: 1.9609 - val_accuracy: 0.1154 - val_loss: 2.1708
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.5000 - loss: 1.8430 - val_accuracy: 0.1154 - val_loss: 2.1764
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - acc

In [182]:
y_hat_old = testmodel.predict(df_o.content)
y_hat_new = testmodel.predict(df_o.content)

Normalizing text
Tokenizing text
Passing text sequences
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
Normalizing text
Tokenizing text
Passing text sequences
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


### With preprocessing and bootstraping

In [183]:
start = time.time()
testmodel.fit(bootstrapping=True, preprocess=True, epochs=20, factor=1000)
end = time.time()
print("Elapsed time: ", np.round((end-start)/60,2),"minutes.")

Bootstrapping by a factor of 1000
Normalizing text
Fitting text tokenizer
Fitting label tokenizer
Tokenizing text
Tokenizing labels
Passing text sequences
Setting model architecture
Compiled model
Epoch 1/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 9s 704ms/step - accuracy: 0.2103 - loss: 2.1276 - val_accuracy: 0.2958 - val_loss: 1.8346
Epoch 2/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 641ms/step - accuracy: 0.3065 - loss: 1.7765 - val_accuracy: 0.3989 - val_loss: 1.4690
Epoch 3/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 755ms/step - accuracy: 0.4529 - loss: 1.4222 - val_accuracy: 0.6975 - val_loss: 1.1072
Epoch 4/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 720ms/step - accuracy: 0.6283 - loss: 1.0896 - val_accuracy: 0.7306 - val_loss: 0.7766
Epoch 5/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 793ms/step - accuracy: 0.7406 - loss: 0.7833 - val_accuracy: 0.9336 - val_loss: 0.4768
Epoch 6/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 726ms/step - accuracy: 0.8900 - loss: 0.5188 - val_accuracy: 1.0000 - val_loss: 0.2934
Epoch 7/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 798ms/st

In [136]:
y_hat_old = testmodel.predict(df_o.content)
y_hat_new = testmodel.predict(df_o.content)

Normalizing text
Tokenizing text
Passing text sequences
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
Normalizing text
Tokenizing text
Passing text sequences
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


## Qualitative Eval

In [137]:
dummy_reviews = [
    "Great Customer support!","Bad customer service!", "Worst Customer support!",#customer support examples
    "Full of hidden fees","Loosing a lot of many with the fees", "Other networks charge less per trade",# fees
     "Too many updates!!", "Cant download app", #Reliability
    "App full of scammers","Danger, lot´s of scammers", "Scams!!", #security
     "Very Easy to use", "Friendly UI", "Fortunately I had no issues so far, the app is very intuitive", #usability
     "Nice features","Great so far", "Excellent", ":)", #generic feedback
     "Great app to loose money!"] #sarcasm
dummy_true_label = [
    "Customer Support", "Customer Support", "Customer Support", 
    "Transaction Fees/Speed","Transaction Fees/Speed","Transaction Fees/Speed",
    "Reliability/Stability", "Reliability/Stability",
    "Security","Security","Security",
    "Usability/UI/UX", "Usability/UI/UX", "Usability/UI/UX",
    "Generic feedback", "Generic feedback", "Generic feedback","Generic feedback",
    "Generic feedback"
]
    
dummy_predictions = testmodel.predict(dummy_reviews)
pd.DataFrame({"dummy_reviews":dummy_reviews, "dummy_true_labels":dummy_true_label,  "predictions":dummy_predictions})

Normalizing text
Tokenizing text
Passing text sequences
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step


,dummy_reviews,dummy_true_labels,predictions
0,Great Customer support!,Customer Support,Generic feedback
1,Bad customer service!,Customer Support,Generic feedback
2,Worst Customer support!,Customer Support,Generic feedback
3,Full of hidden fees,Transaction Fees/Speed,Generic feedback
4,Loosing a lot of many with the fees,Transaction Fees/Speed,Generic feedback
5,Other networks charge less per trade,Transaction Fees/Speed,Generic feedback
6,Too many updates!!,Reliability/Stability,Generic feedback
7,Cant download app,Reliability/Stability,Usability/UI/UX
8,App full of scammers,Security,Usability/UI/UX
9,"Danger, lot´s of scammers",Security,Generic feedback
